In [1]:
class vector:
    N=0;
    Vec=[];
    def __init__(self,vec=None):
        if vec==None or vec ==[]:
            self.N=0
            self.Vec=[]
        else:
            self.Vec=vec
            self.N=len(vec)
    def print_vec(self):
        print(self.Vec)

In [154]:
class square:
    N=0;
    Squ=[];
    def __init__(self,n=0,squ=None):
        if squ==None or squ ==[] :
            self.N=0
            self.Squ=[]
        else:
            self.N=n
            self.Squ=squ
    def print_squ(self):
        for row in self.Squ:
            print(row)
    def transpose(self):
        temp = [[self.Squ[j][i] for j in range(self.N)] for i in range(self.N)]
        return square(self.N,temp)

In [3]:
def matmul(m1,m2):
    if m1.N!=m2.N:
        print("Error")
        return -1
    else:
        tmp1 = []
        for i in range(m1.N):
            tmp2 = []
            for j in range(m1.N):
                tmp3 = 0
                for k in range(m1.N):
                    tmp3+=m1.Squ[i][k]*m2.Squ[k][j]
                tmp2.append(tmp3)
            tmp1.append(tmp2)
        return square(m1.N,tmp1) 

In [4]:
def mvmul(mat,vec):
    if mat.N!=vec.N:
        print("Error")
        return -1
    else:
        tmp1 = []
        for i in range(mat.N):
            tmp2 = 0
            for j in range(mat.N):
                tmp2 += mat.Squ[i][j]*vec.Vec[j]
            tmp1.append(tmp2)
        return vector(tmp1)

In [5]:
def Hilbert_mat(n):
    mat = [[1/(i+j+1) for j in range(n)] for i in range(n)]
    return square(n,mat)

In [124]:
def GEM(mat,vec=None):
    if vec != None and mat.N!=vec.N:
        print("Error")
        return -1
    if vec == None:
        vec = vector([1 for i in range(mat.N)])
    for i in range(mat.N):
        
        maxi = i
        for j in range(mat.N-i):
            if abs(mat.Squ[i+j][i])>abs(mat.Squ[maxi][i]):
                maxi = j+i
        if mat.Squ[maxi][i] == 0:
            return -1
        temp = mat.Squ[maxi]
        mat.Squ[maxi] = mat.Squ[i]
        mat.Squ[i] = temp
        temp = vec.Vec[maxi]
        vec.Vec[maxi] = vec.Vec[i]
        vec.Vec[i] = temp
        
        temp = -1.0/mat.Squ[i][i]
        for j in range(mat.N-i-1):
            l = temp*mat.Squ[i+j+1][i]
            for k in range(mat.N-i):
                mat.Squ[i+j+1][i+k] += l*mat.Squ[i][i+k]
            vec.Vec[i+j+1] += l*vec.Vec[i]
            
    return mat,vec

In [64]:
def Cholesky(mat):
    temp = [[0 for i in range(mat.N)] for j in range(mat.N)]
    for i in range(mat.N):
        if i == 0:
            temp[0][0] = mat.Squ[0][0]**(1/2)
            continue
        for j in range(i):
            temp[i][j] = mat.Squ[i][j]
            for k in range(j):
                temp[i][j] -= temp[i][k]*temp[j][k]
            temp[i][j]/=temp[j][j]
        temp[i][i] = mat.Squ[i][i]
        for j in range(i):
            temp[i][i] -= temp[i][j]**2
        temp[i][i] = temp[i][i]**(1/2)
        
    return square(mat.N,temp)

In [130]:
def upper_tri_solver(mat,vec=None):
    if mat.N!=vec.N and vec != None:
        print("Error")
        return -1
    if vec == None:
        vec = vector([1 for i in range(mat.N)])
    
    temp_vec = []
    for i in range(mat.N):
        if mat.Squ[i][i] == 0:
            print("Cannot Solve!")
            return -1
        temp = vec.Vec[mat.N-1-i]
        for k in range(i):
            temp -= mat.Squ[mat.N-1-i][mat.N-1-k]*temp_vec[k]
        temp /= mat.Squ[mat.N-1-i][mat.N-1-i]
        temp_vec.append(temp)
    temp_vec.reverse()
    return vector(temp_vec)
    

In [148]:
def lower_tri_solver(mat,vec=None):
    if mat.N!=vec.N and vec != None:
        print("Error")
        return -1
    if vec == None:
        vec = vector([1 for i in range(mat.N)])
    
    temp_vec = []
    for i in range(mat.N):
        if mat.Squ[i][i] == 0:
            print("Cannot Solve!")
            return -1
        temp = vec.Vec[i]
        for k in range(i):
            temp -= mat.Squ[i][k]*temp_vec[k]
        temp /= mat.Squ[i][i]
        temp_vec.append(temp)
    return vector(temp_vec)
    

In [156]:
n = 3

In [157]:
#use GEM

m_GEM = Hilbert_mat(n)
v_GEM = vector([1 for i in range(n)])

m_,v_ = GEM(mat=m_GEM,vec=v_GEM)

ans_GEM = upper_tri_solver(m_,v_)

ans_GEM.print_vec()

[3.0000000000000195, -24.000000000000092, 30.00000000000008]


In [160]:
#use Cholesky

m_Cho = Hilbert_mat(n)
v_Cho = vector([1 for i in range(n)])

h = Cholesky(m_Cho)

h_dag = h.transpose()

ans_tmp = lower_tri_solver(h,v_Cho)

ans_Cho = upper_tri_solver(h_dag,ans_tmp)

ans_Cho.print_vec()

[3.000000000000007, -24.00000000000003, 30.00000000000002]


In [198]:
def sqr_newton(s,n=10):
    x = s;
    for i in range(n):
        x = x/2+s/2/x
        
    return x

In [199]:
print(sqr_newton(100))

10.0


In [200]:
sqr_newton(s=5)

2.23606797749979

In [201]:
2.236067977499978**2

5.000000000000843

In [202]:
import time

time_start=time.time()
for i in range(100):
    sqr_newton(i+1)
time_end=time.time()
print('totally cost',time_end-time_start)

totally cost 0.0002300739288330078


In [203]:
import time

time_start=time.time()
for i in range(100):
    (i+1)**(1/2)
time_end=time.time()
print('totally cost',time_end-time_start)

totally cost 7.033348083496094e-05
